# **RuntimeError: Java gateway process exited**

**RuntimeError**: Java gateway process exited before sending its port number when setting spark config.

# Mise en évidence du problème

L'exception se déclenche au moment d'instancier une session Spark, plus spécifiquement sur l'appel de `.getOrCreate()`.

## Reproduction

Le 13 mai, après avoir modifié la variable d'environnement `PYSPARK_SUBMIT_ARGS` pour la valeur en dur `--master local[*] pyspark-shell` plutôt que de l'effectuer programmatiquement, a réglé mon problème.

C'est donc une affaire de chargement, il est un moment où pour changer, il est trop tard.

Pas le temps pour le moment d'investiguer, mais la suite serait :
1. modifier la variable d'environnement système pour la valeur `--master spark://172.28.176.216:7077 --conf spark.driver.host=172.28.176.1` (`SalviaDivinorum`)
2. introduire la modification à la volée avec `os.environ['PYSPARK_SUBMIT_ARGS'] = "--master local[*] pyspark-shell"` et voir si cela reproduit l'erreur.
3. voir ce qu'il se passe si la modification est effectuée au niveau du fichier `.env`

In [4]:
from pyspark.sql import SparkSession
spark = (
    SparkSession
    .builder
    .appName('P8')
    .master('local')
    .config("spark.sql.parquet.writeLegacyFormat", 'true')
    .getOrCreate()
)

## Ce qu'il s'en dit

Quelques références sur le sujet : voir section `RuntimeError: Java gateway` dans `_references.ipynb`.

## Diagnostic

D'abord, les variables d'environnement

In [3]:
from pepper.utils import print_subsubtitle
from pepper.env import show_env
print_subsubtitle("SPARK ENV")
show_env(".*SPARK.*")
print_subsubtitle("JAVA ENV")
show_env(".*JAVA.*")
print_subsubtitle("JAVA ENV")
show_env(".*JAVA.*")
print_subsubtitle("VSCODE ENV")
show_env(".*VSCODE.*")


SPARK ENV
PYSPARK_SUBMIT_ARGS: --master local[*] pyspark-shell
SPARK_HOME: C:\Program Files\Spark

JAVA ENV
JAVA_HOME: C:\Program Files\Java\jdk-17

JAVA ENV
JAVA_HOME: C:\Program Files\Java\jdk-17

VSCODE ENV
VSCODE_AMD_ENTRYPOINT: vs/workbench/api/node/extensionHostProcess
VSCODE_CODE_CACHE_PATH: C:\Users\franc\AppData\Roaming\Code\CachedData\b3e4e68a0bc097f0ae7907b217c1119af9e03435
VSCODE_CRASH_REPORTER_PROCESS_TYPE: extensionHost
VSCODE_CWD: C:\Users\franc\AppData\Local\Programs\Microsoft VS Code
VSCODE_HANDLES_UNCAUGHT_ERRORS: true
VSCODE_IPC_HOOK: \\.\pipe\528ce89b3964f0be503a218713263ba8-1.78.2-main-sock
VSCODE_L10N_BUNDLE_LOCATION: file:///c%3A/Users/franc/.vscode/extensions/ms-ceintl.vscode-language-pack-fr-1.78.2023051009/translations/extensions/vscode.json-language-features.i18n.json
VSCODE_NLS_CONFIG: {"locale":"fr","osLocale":"fr-fr","availableLanguages":{"*":"fr"},"_languagePackId":"464e33a418b005a7ef0102b15075ed48.fr","_translationsConfigFile":"C:\\Users\\franc\\AppData

# Modification de l'environnement

In [ ]:
from fruits.spark_utils import set_master_local
from pepper.env import show_env
import os

set_master_local()
# Récupérer toutes les variables d'environnement
env_vars = os.environ
os.environ['PYSPARK_SUBMIT_ARGS'] = "--master local[*] pyspark-shell"
show_env(".*SPARK.*")

PYSPARK_SUBMIT_ARGS: --master local[*] pyspark-shell
SPARK_HOME: C:\Program Files\Spark
